In [1]:
from bs4 import BeautifulSoup as bsoup
import urllib.robotparser
import requests
import time
import pandas as pd
import numpy as np
import re
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [2]:
# Scrape daily gross for a specific date to test

url = 'https://www.boxofficemojo.com/date/2021-08-08/'
page = requests.get(url)
soup = bsoup(page.text, 'lxml')

all_divs = soup.main.find_all("div", id="table")
all_tables = all_divs[0].find_all("table")
all_trs = all_tables[0].find_all("tr")

for i in range(1, len(all_trs)):
    print(f"Entry row {i} = ")
    
    all_tds = all_trs[i].find_all("td")
    a_href = all_tds[2].find('a',href=True)
    entry = []
    
    for j in range(0, len(all_tds)):
        entry.append(all_tds[j].text)
    
    # go to individual movie page to get genres
    a_href = 'http://boxofficemojo.com' + a_href['href']
    movie_page = requests.get(a_href)
    movie_soup = bsoup(movie_page.text, 'lxml')
    
    table_div = movie_soup.find('div', {'class': 'mojo-summary-values'})
    sub_divs = table_div.find_all('div')

    genres = []
    for s in sub_divs:
        if re.search('^Genres',s.text):
            genres = s.text
            genres = re.sub('Genres', '', genres)
            genres = re.sub('\s+', ' ', genres)
    entry.append(genres.split())
        
    print(entry)

Entry row 1 = 
['1', '1', 'The Suicide Squad', '$5,940,000', '-27.6%', '-', '-', '-', '$26,205,415', '3', 'Warner Bros.\n\n', 'false', 'true', ['Action', 'Adventure', 'Comedy', 'Sci-Fi']]
Entry row 2 = 
['2', '2', 'Jungle Cruise', '$4,616,034', '-30.8%', '-50%', '4,310', '$1,071', '$65,443,044', '10', 'Walt Disney Studios Motion Pictures\n\n', 'false', 'false', ['Action', 'Adventure', 'Comedy', 'Fantasy']]
Entry row 3 = 
['3', '3', 'Old', '$1,213,725', '-28.3%', '-41.4%', '3,138', '$386', '$38,498,750', '17', 'Universal Pictures\n\n', 'false', 'false', ['Drama', 'Mystery', 'Thriller']]
Entry row 4 = 
['4', '4', 'Black Widow', '$1,082,915', '-35.1%', '-42.3%', '3,100', '$349', '$174,280,036', '31', 'Walt Disney Studios Motion Pictures\n\n', 'false', 'false', ['Action', 'Adventure', 'Sci-Fi']]
Entry row 5 = 
['5', '5', 'Stillwater', '$823,955', '-32.9%', '-41.1%', '2,611', '$315', '$9,968,760', '10', 'Focus Features\n\n', 'false', 'false', ['Crime', 'Drama', 'Thriller']]
Entry row 6 = 
[

In [3]:
# Build date for URL and make requests for each day from 2015-2019

month_and_days = {'01': [x for x in range(1,32)], '02': [x for x in range(1,29)], '03':[x for x in range(1,32)],
                  '04':[x for x in range(1,31)],'05':[x for x in range(1,32)], '06':[x for x in range(1,31)], 
                 '07':[x for x in range(1,32)], '08':[x for x in range(1,32)], '09':[x for x in range(1,31)],
                 '10': [x for x in range(1,32)], '11':[x for x in range(1,31)], '12':[x for x in range(1,32)]}

leap_years = [2008, 2016, 2020]
leap_year_month_days = {'01': [x for x in range(1,32)], '02': [x for x in range(1,30)], '03':[x for x in range(1,32)],
                  '04':[x for x in range(1,31)],'05':[x for x in range(1,32)], '06':[x for x in range(1,31)], 
                 '07':[x for x in range(1,32)], '08':[x for x in range(1,32)], '09':[x for x in range(1,31)],
                 '10': [x for x in range(1,32)], '11':[x for x in range(1,31)], '12':[x for x in range(1,32)]}

all_data = []

for year in range(2015, 2020):
    # build url for each year-month-day and scrape table data
    url = 'https://www.boxofficemojo.com/date/'
    
    if year in leap_years:
        m_d = leap_year_month_days
    else:
        m_d = month_and_days
        
    y = str(year)
    
    for month in m_d:
        m = '-' + month
        for day in m_d[month]:
            if len(str(day)) == 1:
                d = '-' + '0' + str(day)
            else:
                d = '-' + str(day)
            
            date = y + m + d
            url_curr = url + date
            
            #get data
            page = requests.get(url_curr)
            soup = bsoup(page.text, 'lxml')
            
            all_divs = soup.main.find_all("div", id="table")
            
            try:
                all_tables = all_divs[0].find_all("table")
            except:
                print(date)
            
            try:
                all_trs = all_tables[0].find_all("tr")
            except:
                print(date)
            
            for i in range(1, len(all_trs)):

                all_tds = all_trs[i].find_all("td")
                a_href = all_tds[2].find('a',href=True)
                a_href = 'http://boxofficemojo.com' + a_href['href']
                entry = []

                for j in range(0, len(all_tds)):
                    entry.append(all_tds[j].text)       
                
                entry = entry[0:11]
                entry.append(a_href)
                entry.append(date)

            
                all_data.append(entry)
    print(f'{year} processed...')

2015 processed...
2016 processed...
2017 processed...
2018 processed...
2019 processed...


In [4]:
len(all_data)

95953

In [5]:
column_names = ['TD', 'YD', 'Movie', 'Daily', '%YD', '%LW', 'Theaters', 'Avg', 'To_Date', 
                'Days', 'Distributor', 'href','Date']
df = pd.DataFrame(all_data, columns=column_names)
df.to_pickle('boxofficemojo-daily_gross.pkl')
df

,TD,YD,Movie,Daily,%YD,%LW,Theaters,Avg,To_Date,Days,Distributor,href,Date
0,1,1,The Hobbit: The Battle of the Five Armies,"$9,320,601",+55.3%,-29%,"3,875","$2,405","$198,869,927",16,Warner Bros.\n\n,http://boxofficemojo.com/release/rl2354677249/...,2015-01-01
1,2,3,Into the Woods,"$8,081,110",+52.7%,-46.4%,"2,507","$3,223","$72,143,214",8,Walt Disney Studios Motion Pictures\n\n,http://boxofficemojo.com/release/rl5932545/?re...,2015-01-01
2,3,2,Unbroken,"$6,953,045",+17.4%,-55%,"3,167","$2,195","$69,481,655",8,Universal Pictures\n\n,http://boxofficemojo.com/release/rl3212215809/...,2015-01-01
3,4,4,Night at the Museum: Secret of the Tomb,"$5,974,323",+22.5%,-16.4%,"3,914","$1,526","$75,328,905",14,Twentieth Century Fox\n\n,http://boxofficemojo.com/release/rl2087159297/...,2015-01-01
4,5,5,Annie,"$4,257,057",+42%,-6.8%,"3,197","$1,331","$61,208,977",14,Sony Pictures Entertainment (SPE)\n\n,http://boxofficemojo.com/release/rl641893889/?...,2015-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95948,50,47,Hustlers,$185,-76.8%,-22.6%,3,$61,"$104,960,643",110,STX Entertainment\n\n,http://boxofficemojo.com/release/rl3255404033/...,2019-12-31
95949,51,52,Midnight Family,$134,-45.1%,-2.2%,2,$67,"$15,537",26,1091 Pictures\n\n,http://boxofficemojo.com/release/rl2601223681/...,2019-12-31
95950,52,53,Kind Hearts and Coronets2019 Re-release,$108,-53.6%,-,1,$108,"$27,523",35,Rialto Pictures\n\n,http://boxofficemojo.com/release/rl1923056129/...,2019-12-31
95951,53,50,Sympathy for the Devil,$73,-77.8%,-24%,1,$73,"$45,927",33,-,http://boxofficemojo.com/release/rl3399975425/...,2019-12-31


In [6]:
df = pd.read_pickle('boxofficemojo-daily_gross.pkl')

In [7]:
titles_unique = list(set(df['Movie']))
len(titles_unique)

2353

In [8]:
# Go to each movie page to scrape genre data and add to dataframe
genre_dict = {}

for m in titles_unique:
    h = list(df.loc[df['Movie'] == m, 'href'])[0]
    movie_page = requests.get(h)
    movie_soup = bsoup(movie_page.text, 'lxml')

    table_div = movie_soup.find('div', {'class': 'mojo-summary-values'})
    sub_divs = table_div.find_all('div')

    for s in sub_divs:
        if re.search('^Genres',s.text):
            genres = s.text
            genres = re.sub('Genres', '', genres)
            genres = re.sub('\s+', ' ', genres)
    genre_dict[m] = genres.split()

genre_dict

{'The Star': ['Adventure', 'Animation', 'Comedy', 'Family', 'Fantasy'],
 'High-Rise': ['Drama', 'Sci-Fi'],
 'The Dark Tower': ['Action', 'Adventure', 'Fantasy', 'Sci-Fi', 'Western'],
 'Paths of the Soul': ['Drama'],
 'Harry  Potter IMAX Marathon': ['Drama'],
 'Hail, Caesar!': ['Comedy', 'Drama', 'Music', 'Mystery'],
 'The Dawn Wall': ['Biography', 'Documentary', 'Sport'],
 'BTS World Tour: Love Yourself in Seoul': ['Documentary', 'Music'],
 'Sui Dhaaga': ['Comedy', 'Drama'],
 'The Night Is Short, Walk on Girl': ['Adventure',
  'Animation',
  'Comedy',
  'Family',
  'Fantasy',
  'Romance'],
 'The Wandering Soap Opera': ['Comedy', 'Drama', 'Fantasy'],
 'Bumblebee': ['Action', 'Adventure', 'Sci-Fi'],
 'Status Update': ['Comedy', 'Fantasy', 'Romance'],
 'Paulette2015 Re-release': ['Comedy', 'Crime'],
 'The Little Mermaid': ['Adventure', 'Drama', 'Fantasy', 'Romance'],
 'Kirk Cameron: Connect': ['Documentary'],
 'Ophelia': ['Drama', 'Romance', 'Thriller'],
 'Table 19': ['Comedy', 'Drama', '

In [9]:
# Check
len(genre_dict)

2353

In [11]:
# For each href in the dictionary, unpack genres into string and add to genre column for that film
for m in genre_dict:
    genres = genre_dict[m]
    genre_str = ''
    for g in genres:
        genre_str += g + '|'
    df.loc[df['Movie'] == m,'genre'] = genre_str

df.genre.sample(50)

18396                           Action|Drama|Thriller|War|
61083                                      Comedy|Romance|
87392                                               Drama|
66151                   Biography|Documentary|Drama|Music|
86541                                        Comedy|Drama|
54814                              Action|Sci-Fi|Thriller|
62216                        Biography|Drama|Family|Music|
95156                                        Comedy|Crime|
58362              Biography|Crime|Drama|Mystery|Thriller|
63013                         Comedy|Music|Romance|Sci-Fi|
82204                                     Biography|Drama|
60551                                Comedy|Drama|Romance|
83605                             Action|Adventure|Sci-Fi|
37657                     Animation|Comedy|Family|Musical|
77835                                Comedy|Crime|Mystery|
46610                      Action|Adventure|Comedy|Sci-Fi|
34                                            Crime|Dram

In [12]:
unique_genre =  df.genre.str.split('|').sum()
unique_genre = set(unique_genre)
unique_genre

for g in unique_genre:
    df[g] = df.genre.map( lambda x: g in x.split('|') )
    
df.sample(10)

,TD,YD,Movie,Daily,%YD,%LW,Theaters,Avg,To_Date,Days,...,Animation,History,Sport,Drama,Film-Noir,Western,War,Music,Fantasy,Short
35316,48,49,The Lion in Winter2016 Re-release,$166,+80.4%,-89.8%,1,$166,"$16,907",16,...,False,True,False,True,False,False,False,False,False,False
23273,51,45,Harmony,$129,-79.6%,-,1,$129,"$23,248",5,...,True,False,False,False,False,False,False,False,False,False
71058,24,27,Tea With the Dames,"$12,713",-54.2%,-9.3%,71,$179,"$448,001",32,...,False,False,False,False,False,False,False,False,False,False
73727,14,14,The Favourite,"$101,684",+5.6%,+149.4%,34,"$2,990","$1,939,514",13,...,False,True,False,True,False,False,False,False,False,False
32580,31,32,Snowden,"$9,616",+34%,-29.6%,115,$83,"$21,496,925",54,...,False,False,False,True,False,False,False,False,False,False
59959,21,16,Game Night,"$224,575",-56.9%,-67%,804,$279,"$67,216,416",45,...,False,False,False,False,False,False,False,False,False,False
5017,32,28,The Lazarus Effect,"$10,856",+100.3%,-59.7%,74,$146,"$25,462,601",50,...,False,False,False,False,False,False,False,False,False,False
51691,6,7,Blade Runner 2049,"$705,678",-29.9%,-45.5%,"1,464",$482,"$85,542,502",31,...,False,False,False,True,False,False,False,False,False,False
44682,14,13,Rough Night,"$143,564",-29.1%,-63.6%,"1,657",$86,"$21,233,202",21,...,False,False,False,False,False,False,False,False,False,False
87125,45,49,Meeting Gorbachev,"$1,168","+1,342%",+961.8%,4,$292,"$243,762",85,...,False,True,False,False,False,False,False,False,False,False


In [13]:
df.genre.isnull().sum()

0

In [14]:
# Replace non-numeric values for numeric data columns
df['Daily'] = df['Daily'].str.replace('[^0-9]', '')
df['To_Date'] = df['To_Date'].str.replace('[^0-9]', '')
df['%YD'] = df['%YD'].str.replace('+', '')
df['%YD'] = df['%YD'].str.replace('-', '')
df['%YD'] = df['%YD'].str.replace('%', '')
df['%YD'] = df['%YD'].str.replace(',', '')
df['%YD'] = df['%YD'].str.replace('<0.1', '0')
df['%LW'] = df['%LW'].str.replace('+', '')
df['%LW'] = df['%LW'].str.replace('-', '')
df['%LW'] = df['%LW'].str.replace('%', '')
df['%LW'] = df['%LW'].str.replace(',', '')
df['%LW'] = df['%LW'].str.replace('<0.1', '0')
df['Theaters'] = df['Theaters'].str.replace('[^0-9]', '')
df['Avg'] = df['Avg'].str.replace('[^0-9]', '')
df['Days'] = df['Days'].str.replace('[^0-9]', '')
df['Distributor'] = df['Distributor'].str.replace('\\n', '')
df.head()

,TD,YD,Movie,Daily,%YD,%LW,Theaters,Avg,To_Date,Days,...,Animation,History,Sport,Drama,Film-Noir,Western,War,Music,Fantasy,Short
0,1,1,The Hobbit: The Battle of the Five Armies,9320601,55.3,29,3875,2405,198869927,16,...,False,False,False,False,False,False,False,False,True,False
1,2,3,Into the Woods,8081110,52.7,46.4,2507,3223,72143214,8,...,False,False,False,True,False,False,False,False,True,False
2,3,2,Unbroken,6953045,17.4,55,3167,2195,69481655,8,...,False,False,True,True,False,False,True,False,False,False
3,4,4,Night at the Museum: Secret of the Tomb,5974323,22.5,16.4,3914,1526,75328905,14,...,False,False,False,False,False,False,False,False,True,False
4,5,5,Annie,4257057,42,6.8,3197,1331,61208977,14,...,False,False,False,True,False,False,False,False,False,False


In [15]:
# Update numeric data types
df['Daily'] = pd.to_numeric(df['Daily'])
df['%YD'] = pd.to_numeric(df['%YD'])
df['%LW'] = pd.to_numeric(df['%LW'])
df['Theaters'] = pd.to_numeric(df['Theaters'])
df['Avg'] = pd.to_numeric(df['Avg'])
df['To_Date'] = pd.to_numeric(df['To_Date'])
df['Days'] = pd.to_numeric(df['Days'])
df.dtypes

TD              object
YD              object
Movie           object
Daily            int64
%YD            float64
%LW            float64
Theaters       float64
Avg            float64
To_Date        float64
Days           float64
Distributor     object
href            object
Date            object
genre           object
                  bool
Comedy            bool
Romance           bool
Sci-Fi            bool
Adult             bool
Mystery           bool
Documentary       bool
Adventure         bool
News              bool
Biography         bool
Crime             bool
Thriller          bool
Family            bool
Action            bool
Musical           bool
Horror            bool
Animation         bool
History           bool
Sport             bool
Drama             bool
Film-Noir         bool
Western           bool
War               bool
Music             bool
Fantasy           bool
Short             bool
dtype: object

In [16]:
df['Date'] = pd.to_datetime(df['Date'])

In [17]:
df.sample(20)

,TD,YD,Movie,Daily,%YD,%LW,Theaters,Avg,To_Date,Days,...,Animation,History,Sport,Drama,Film-Noir,Western,War,Music,Fantasy,Short
53075,33,32,Happy Death Day,4390,2.6,85.6,131.0,33.0,55579925.0,47.0,...,False,False,False,False,False,False,False,False,False,False
63157,10,12,A Quiet Place,237002,49.9,66.0,1156.0,205.0,183757371.0,60.0,...,False,False,False,True,False,False,False,False,False,False
2736,11,11,Paddington,125399,23.1,70.4,1837.0,68.0,68243149.0,41.0,...,True,False,False,False,False,False,False,False,True,False
56815,54,53,Signature Move,101,77.4,NaN,1.0,101.0,30900.0,136.0,...,False,False,False,True,False,False,False,False,False,False
29693,46,46,Other People,3295,29.6,20.5,12.0,274.0,80330.0,17.0,...,False,False,False,True,False,False,False,False,False,False
11521,43,44,Magic Mike XXL,4046,29.7,74.8,55.0,73.0,65990414.0,75.0,...,False,False,False,True,False,False,False,True,False,False
70907,60,38,Bigger,173,94.8,98.4,5.0,34.0,45216.0,8.0,...,False,False,True,True,False,False,False,False,False,False
17965,1,1,Kung Fu Panda 3,2291034,30.9,NaN,3955.0,579.0,45323814.0,5.0,...,True,False,False,False,False,False,False,False,True,False
32782,14,13,Moonlight,521334,23.1,0.3,176.0,2962.0,4362451.0,23.0,...,False,False,False,True,False,False,False,False,False,False
24739,37,37,The Fallen Idol2016 Re-release,282,9.3,68.1,2.0,141.0,46477.0,28.0,...,False,False,False,True,False,False,False,False,False,False


In [18]:
df.dtypes

TD                     object
YD                     object
Movie                  object
Daily                   int64
%YD                   float64
%LW                   float64
Theaters              float64
Avg                   float64
To_Date               float64
Days                  float64
Distributor            object
href                   object
Date           datetime64[ns]
genre                  object
                         bool
Comedy                   bool
Romance                  bool
Sci-Fi                   bool
Adult                    bool
Mystery                  bool
Documentary              bool
Adventure                bool
News                     bool
Biography                bool
Crime                    bool
Thriller                 bool
Family                   bool
Action                   bool
Musical                  bool
Horror                   bool
Animation                bool
History                  bool
Sport                    bool
Drama     

In [19]:
# Send data to pickle
df.to_pickle('clean-boxofficemojo-daily_gross.pkl')